Here, we shall write a quick script using Tweepy to gather the followers of several Twitter users. First, we need to grab our Twitter API credidentials.

In [1]:
import tweepy
import dill

keys = dill.load(open('.secrets/api-keys.pkd', 'rb'))

auth = tweepy.OAuthHandler(keys['API'], keys['API secret'])

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [4]:
import time
yang_followers = []
for page in tweepy.Cursor(api.followers, screen_name="AndrewYang").pages():
    yang_followers.extend(page)

Rate limit reached. Sleeping for: 889
Rate limit reached. Sleeping for: 890
Rate limit reached. Sleeping for: 891
Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 892
Rate limit reached. Sleeping for: 891
Rate limit reached. Sleeping for: 892
Rate limit reached. Sleeping for: 892
Rate limit reached. Sleeping for: 892
Rate limit reached. Sleeping for: 892
Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 891
Rate limit reached. Sleeping for: 892
Rate limit reached. Sleeping for: 892
Rate limit reached. Sleeping for: 892
Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 892
Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 892
Rate limit reached. Sleeping for: 892
Rate limit reached. Sleeping for: 892
Rate limit reached. Sleeping for: 891
Rate limit reached. Sleeping for: 892
Rate limit r

KeyboardInterrupt: 

In [5]:
len(yang_followers)

36000

In [2]:
yang_followers = dill.load(open('../scrapped_data/twitter-data/yang_followers.pkd', 'rb'))

In [12]:
names = []
screen_names = []
locations = []
descriptions = []
follower_counts = []
friends_counts = []

for follower in yang_followers:
    names.append(follower.name)
    screen_names.append(follower.screen_name)
    locations.append(follower.location)
    descriptions.append(follower.description)
    follower_counts.append(follower.followers_count)
    friends_counts.append(follower.friends_count)

In [43]:
import pandas as pd

df = pd.DataFrame({'name': names, 
                   'screen_name': screen_names, 
                   'location': locations, 
                   'description': descriptions, 
                   'followers_count': follower_counts, 
                   'friends_count': friends_counts})

In [44]:
from collections import defaultdict

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

states_full = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
               "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
            "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
            "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
            "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
            "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
            "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
            "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

state_counts = defaultdict(int)

for location in df.location.values:
    for state in states:
        if state in location:
            state_counts[state] += 1
    for i, state in enumerate(states_full):
        if state in location:
            state_counts[states[i]] += 1
            
df_state_counts = pd.DataFrame(state_counts)

ValueError: If using all scalar values, you must pass an index

In [54]:
ordered_count = sorted([(k, v) for k, v in state_counts.items()], key=lambda x: states.index(x[0]))

In [58]:
df_pop = pd.read_csv('../misc_data/state_populations.csv')

In [69]:
df_pop[df_pop['State'] == 'California'].Percent.values

array([0.12125884])

In [73]:
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import pandas as pd
import numpy as np

state_counts

for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

followers_by_state = np.array([location[0] for location in ordered_count])
population_by_state = np.array([df_pop[df_pop['State'] == states_full[states.index(location)]].Percent.values for location in ordered_count])

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = [location[0] for location in ordered_count],
        z = [location[1] for location in ordered_count],
#         locations = df['code'],
#         z = df['total exports'].astype(float),
        locationmode = 'USA-states',
        text = [location[0] for location in ordered_count],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = "Millions USD"
        )
    ) ]

layout = dict(
        title = 'Number of Followers by State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
        ),
    )

fig = dict( data=data, layout=layout )
init_notebook_mode(connected=True)
iplot(fig)

ValueError: ('AL', 109) is not in list

In [48]:
len(tweets)

3192

In [49]:
tweet_text = []
tweet_dates = []
for tweet in tweets:
    tweet_text.append(tweet._json['full_text'])
    tweet_dates.append(tweet._json['created_at'])

In [62]:
import re

http_tail = re.compile("https:.*$")

tweet = tweet_text[1]
tweet_text = [re.sub(http_tail, '', text) for text in tweet_text]

In [65]:
tweet_text[2]

'So, it has now been determined, by 18 people that truly hate President Trump, that there was No Collusion with Russia. In fact, it was an illegal investigation that should never have been allowed to start. I fought back hard against this Phony &amp; Treasonous Hoax!'

In [66]:
import pandas as pd

df = pd.DataFrame({'Text': tweet_text, 'Date': tweet_dates})

In [67]:
df

,Text,Date
0,"THANK YOU Texas, I love you!",Wed Apr 10 21:41:06 +0000 2019
1,"Join me in Crosby, Texas as I deliver remarks ...",Wed Apr 10 21:37:40 +0000 2019
2,"So, it has now been determined, by 18 people t...",Wed Apr 10 19:45:55 +0000 2019
3,Spoke to Bibi @Netanyahu to congratulate him o...,Wed Apr 10 19:17:19 +0000 2019
4,Trump flags being waived at the Bibi @Netanyah...,Wed Apr 10 14:48:02 +0000 2019
5,RT @JudicialWatch: Judicial Watch President @T...,Wed Apr 10 03:02:43 +0000 2019
6,RT @JudicialWatch: BREAKING: JW announced toda...,Wed Apr 10 02:59:40 +0000 2019
7,"Everybody is now acknowledging that, right fro...",Wed Apr 10 02:22:07 +0000 2019
8,"RT @IWV: .@POTUS @realDonaldTrump has ""deliver...",Wed Apr 10 02:05:47 +0000 2019
9,RT @HeyTammyBruce: Trump Job Approval Jumps to...,Wed Apr 10 02:00:46 +0000 2019


In [57]:
len(tweet_text)

100

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation 

n_features = 500

tfidf_vectorizer = TfidfVectorizer(max_df=0.99, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(tweet_text)

In [71]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')

In [73]:
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [74]:
tf = tf_vectorizer.fit_transform(tweet_text)

In [76]:
n_components = 10
n_top_words = 10

lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)

tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0: united states year court looking supreme tonight end kavanaugh foxnews
Topic #1: rt realdonaldtrump today president trump american great amp whitehouse people
Topic #2: fbi meeting hillary new crooked clinton york times comey amp
Topic #3: great thank state vote house republican congratulations new florida governor
Topic #4: news fake amp media collusion witch hunt russia trump mueller
Topic #5: border wall democrats security country amp want people stop southern
Topic #6: trade china countries deal dollars tariffs world farmers billions nato
Topic #7: president just people law trump did like don time 000
Topic #8: north soon good korea jobs better forward look level way
Topic #9: great america strong crime make total endorsement military job vets


In [65]:
import dill

dill.dump(tweets, open('tweets-1.pkb', 'wb'))

In [77]:
tweets = []
for page in range(1, 16 + 1):
    tweets = tweets + api.user_timeline('HillaryClinton', count=200, tweet_mode='extended', page=page)
    time.sleep(75)

In [78]:
dill.dump(tweets, open('clinton-tweets.pkd', 'wb'))

In [79]:
tweet_text = []
tweet_dates = []
for tweet in tweets:
    tweet_text.append(tweet._json['full_text'])
    tweet_dates.append(tweet._json['created_at'])

In [80]:
http_tail = re.compile("https:.*$")

tweet = tweet_text[1]
tweet_text = [re.sub(http_tail, '', text) for text in tweet_text]

In [81]:
df = pd.DataFrame({'Text': tweet_text, 'Date': tweet_dates})

In [83]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.99, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(tweet_text)

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(tweet_text)

lda.fit(tf)

tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0: ve family night got best 000 stand military let thanks
Topic #1: people young future year million hfa man woman new old
Topic #2: watch america world live tax matter white great tune class
Topic #3: trump donald rt hillary president debate just timkaine campaign like
Topic #4: make vote women rights sure america today college health plan
Topic #5: families children kids states working need help administration united chip
Topic #6: day join thank candidates office today new election gun friend
Topic #7: trump say change climate pence democracy clear president won need
Topic #8: amp years hillary going country days care just let work
Topic #9: rt hillary clinton hillaryclinton president flotus vote good potus candidate
